<a href="https://colab.research.google.com/github/kalai2315/Langchain_for_AgentiAI/blob/main/Creating_Agent_Tools_and_Tool_Calling_by_LLMs_with_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
%pip install -U --quiet langchain langchain-google-genai langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [3]:
# to create custom tools
!pip install wikipedia==1.4.0
!pip install markitdown
# to highlight json
!pip install rich

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=fcc8d302ae175414ca21c731fecad9c7c2354375d805f12f9c8183d0854c56fc
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 97.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.4 MB/s eta 0:00:00


In [4]:

import os
from getpass import getpass

os.environ["GOOGLE_API_KEY"] = getpass("Enter your Google API Key: ")

Enter your Google API Key: ··········


In [5]:

import os
from getpass import getpass

os.environ["TAVILY_API_KEY"] = getpass("Enter your tavily API Key: ")

Enter your tavily API Key: ··········


In [6]:

import os
from getpass import getpass

os.environ["WEATHER_API_KEY"] = getpass("Enter your weather API Key: ")

Enter your weather API Key: ··········


In [7]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wiki_api_wrapper = WikipediaAPIWrapper(top_k_results=3,
                                       doc_content_chars_max=8000)
wiki_tool = WikipediaQueryRun(api_wrapper=wiki_api_wrapper, features="lxml")

In [8]:
wiki_tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [9]:
wiki_tool.args

{'query': {'description': 'query to look up on wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [10]:
print(wiki_tool.invoke({"query": "Microsoft"}))

Page: Microsoft
Summary: Microsoft Corporation is an American multinational corporation and technology conglomerate headquartered in Redmond, Washington. Founded in 1975, the company became influential in the rise of personal computers through software like Windows, and the company has since expanded to Internet services, cloud computing, video gaming and other fields. Microsoft is the largest software maker, one of the most valuable public U.S. companies, and one of the most valuable brands globally.
Microsoft was founded by Bill Gates and Paul Allen to develop and sell BASIC interpreters for the Altair 8800. It rose to dominate the personal computer operating system market with MS-DOS in the mid-1980s, followed by Windows. During the 41 years from 1980 to 2021 Microsoft released 9 versions of MS-DOS with a median frequency of 2 years, and 13 versions of Windows with a median frequency of 3 years. The company's 1986 initial public offering (IPO) and subsequent rise in its share price 

In [11]:
print(wiki_tool.invoke({"query": "AI"}))

Page: Artificial intelligence
Summary: Artificial intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals.
High-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); virtual assistants (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., language models and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go). However, many AI applications are not perceived as AI: "A lot of cutting edge AI has filtered into general 

In [12]:
from langchain.agents import Tool

wiki_tool_init = Tool(name="Wikipedia",
                      func=wiki_api_wrapper.run,
                      description="useful when you need a detailed answer about general knowledge")

In [13]:
wiki_tool_init.description

'useful when you need a detailed answer about general knowledge'

In [14]:
wiki_tool_init.args

{'tool_input': {'type': 'string'}}

In [15]:
print(wiki_tool_init.invoke({"tool_input": "AI"}))

Page: Artificial intelligence
Summary: Artificial intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals.
High-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); virtual assistants (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., language models and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go). However, many AI applications are not perceived as AI: "A lot of cutting edge AI has filtered into general 

In [17]:
from langchain_community.tools.tavily_search import TavilySearchResults

tavily_tool = TavilySearchResults(max_results=8,
                                search_depth='advanced',
                                include_raw_content=True)

/tmp/ipython-input-17-2367757474.py:3: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tavily_tool = TavilySearchResults(max_results=8,


In [18]:
tavily_tool.args

{'query': {'description': 'search query to look up',
  'title': 'Query',
  'type': 'string'}}

In [19]:
tavily_tool.description

'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.'

In [20]:
results = tavily_tool.invoke("Tell me about Microsoft")
results

[{'title': 'Microsoft Corporation | History, Products, & Facts | Britannica Money',
  'url': 'https://www.britannica.com/money/Microsoft-Corporation',
  'content': 'Microsoft Corporation is an American multinational technology company that has been a key player in the computer industry since its founding in 1975. Headquartered in Redmond, Washington, Microsoft initially gained dominance in the operating systems market during the 1980s and ’90s with MS-DOS and Windows. Its strong market position eventually gave rise to the company’s diverse and extensive range of technological offerings, from personal computing and entertainment to enterprise cloud [...] By the mid-1990s Microsoft, which became a publicly owned corporation in 1986, had become one of the most powerful brands and profitable companies in American history. It consistently earned profits of 25 cents on every sales dollar, an astonishing record. In the company’s 1996 fiscal year, it topped $2 billion in net income for the fir

In [21]:
from langchain_core.tools import tool

@tool
def multiply(a, b):
    """Multiply two numbers."""
    return a * b


# Let's inspect some of the attributes associated with the tool.
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers.
{'a': {'title': 'A'}, 'b': {'title': 'B'}}


In [22]:
type(multiply)

langchain_core.tools.structured.StructuredTool

In [23]:
multiply.invoke({"a": 2, "b": 3})

6

In [24]:
multiply.invoke({"a": 2.1, "b": 3.2})

6.720000000000001

In [25]:
multiply.invoke({"a": 2, "b": 'abc'})

'abcabc'

In [26]:
from pydantic import BaseModel, Field
from langchain_core.tools import StructuredTool

class CalculatorInput(BaseModel):
    a: float = Field(description="first number")
    b: float = Field(description="second number")


def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b

# we could also use the @tool decorator from before
multiply = StructuredTool.from_function(
    func=multiply,
    name="multiply",
    description="use to multiply numbers",
    args_schema=CalculatorInput,
    return_direct=True
    )

# Let's inspect some of the attributes associated with the tool.
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
use to multiply numbers
{'a': {'description': 'first number', 'title': 'A', 'type': 'number'}, 'b': {'description': 'second number', 'title': 'B', 'type': 'number'}}


In [27]:
multiply.invoke({"a": 2, "b": 3})

6.0

In [29]:
#this code will error out as abc is not a floating point number
multiply.invoke({"a": 2, "b": 'abc'})

ValidationError: 1 validation error for CalculatorInput
b
  Input should be a valid number, unable to parse string as a number [type=float_parsing, input_value='abc', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/float_parsing

# **Build a Web Search & Information Extraction Tool**

In [30]:
tavily_tool = TavilySearchResults(max_results=5,
                                  search_depth='advanced',
                                  include_raw_content=True)

result = tavily_tool.invoke("Tell me about Microsoft's Q4 2024 earning call report")
result

[{'title': 'Microsoft Q4 2024 Earnings Call Transcript',
  'url': 'https://www.marketbeat.com/earnings/reports/2024-7-30-microsoft-co-stock/',
  'content': 'Next, the Intelligent Cloud segment. Revenue was $28,500,000,000 increasing 19% 20% in constant currency, in line with expectations. Overall, server products and cloud services revenue grew 21% 22% in constant currency. Azure and other cloud Services revenue grew 29% 30% in constant currency, in line with expectations and consistent with Q3 when adjusting for leap year. Azure growth included 8 points from AI Services where demand remained higher than our available capacity. [...] Next, the Intelligent Cloud segment. Revenue was $28,500,000,000 increasing 19% 20% in constant currency, in line with expectations. Overall, server products and cloud services revenue grew 21% 22% in constant currency. Azure and other cloud Services revenue grew 29% 30% in constant currency, in line with expectations and consistent with Q3 when adjusting 

In [31]:
result[0]['url']

'https://www.marketbeat.com/earnings/reports/2024-7-30-microsoft-co-stock/'

In [32]:
from markitdown import MarkItDown

md = MarkItDown()
doc_content = md.convert(result[0]['url'])

In [35]:
doc_content

In [42]:
doc_content = md.convert(result[0]['url'])

In [37]:
print(doc_content.title.strip())

Moomoo SG - Invest Smarter with One Super App


In [43]:
print(doc_content.text_content)

[Skip to main content](#main)

[![MarketBeat home page](/images/master/MarketBeat-logo-r-white.svg?v=2019)](https://www.marketbeat.com "MarketBeat")

* [Research Tools](/all-access/)
  + [All Access Research Tools](/all-access/)
    - [Live News Feed](/all-access/live-news/)
    - [Momentum Alerts](/manage/momentum-alerts/)
    - [Idea Engine](/all-access/idea-engine/)
    - [Export Data (CSV)](/all-access/export-data/)
    - [See All Research Tools](/all-access/)
  + [My MarketBeat](/manage/watchlists/)
    - [My Portfolio](/manage/watchlists/)
    - [My Newsletter](/manage/watchlists/#newsletter)
    - [My Account](/manage/)
  + [Calculators](/calculators/)
    - [Dividend Calculator](/dividends/calculator/)
    - [Dividend Yield Calculator](/dividends/yield-calculator/)
    - [Market Cap Calculator](/calculators/market-cap-calculator/)
    - [Options Profit Calculator](/calculators/options-profit-calculator/)
    - [Stock Average Calculator](/calculators/stock-average-calculator/)
 

In [44]:
from markitdown import MarkItDown
from langchain_community.tools.tavily_search import TavilySearchResults
from tqdm import tqdm
import requests

tavily_tool = TavilySearchResults(max_results=5,
                                  search_depth='advanced',
                                  include_answer=False,
                                  include_raw_content=True)
md = MarkItDown()

@tool
def search_web_extract_info(query: str) -> list:
    """Search the web for a query and extracts useful information from the search links"""
    results = tavily_tool.invoke(query)
    docs = []
    for result in tqdm(results):
        # Extracting all text content from the URL
        try:
            extracted_info = md.convert(result['url'])
            text_title = extracted_info.title.strip()
            text_content = extracted_info.text_content.strip()
            docs.append(text_title + '\n' + text_content)
        except:
            print('Extraction blocked for url: ', result['url'])
            pass

    return docs

In [45]:
docs = search_web_extract_info('OpenAI GPT-4o')

/tmp/ipython-input-45-1285904659.py:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = search_web_extract_info('OpenAI GPT-4o')
100%|██████████| 5/5 [00:02<00:00,  1.99it/s]


In [46]:
from IPython.display import display, Markdown

display(Markdown(docs[0]))

OpenAI’s GPT-4o: Everything you need to know in one place
![close icon](https://cdn.prod.website-files.com/5e0a5d9d743608d0f3ea6753/660181cd9172335cd80c4ad9__Mini%20X_.svg)

![daily.dev platform](https://cdn.prod.website-files.com/5e0a5d9d743608d0f3ea6753/660180848a3e21c9ad344747_daily.dev%20Feed%20-%201280x560%20(1).png)

#### Discover more from daily.dev

Personalized news feed, dev communities and search, much better than what’s out there. Maybe ;)

[**Start reading** - Free forever](https://play.google.com/store/apps/details?id=dev.daily)[**Start reading** - Free forever](https://api.daily.dev/get)

Continue reading >

[![Daily Logo](https://cdn.prod.website-files.com/5e0a5d9d743608d0f3ea6753/5f1d8b1b9c7814aae6b69044_Daily%20Full%20logo.svg)](/?r=0)[Start reading **- Free forever**](https://api.daily.dev/get)[Start now](https://app.daily.dev/?utm_source=landing&utm_medium=cta&utm_campaign=landing_conversions)

![Hamburger](https://cdn.prod.website-files.com/5e0a5d9d743608d0f3ea6753/5e1c903d4b6c08145d84ed87_Hamburger.svg)

[Home](/?r=0)![arrow-down](https://cdn.prod.website-files.com/5e0a5d9d743608d0f3ea6753/60aa1092c2681854cb7d9e94_Arrow%20Down.svg)[Blog](/blog-superold)![arrow-down](https://cdn.prod.website-files.com/5e0a5d9d743608d0f3ea6753/60aa1092c2681854cb7d9e94_Arrow%20Down.svg)[Get into tech](/categories/get-into-tech)

# OpenAI’s GPT-4o: Everything you need to know in one place

May 14, 2024

![OpenAI’s GPT-4o: Everything you need to know in one place](https://cdn.prod.website-files.com/5e0f1144930a8bc8aace526c/66437e8ee15f54e26397222c_664375a4664bfde22f5257a7-8d73159ac297fb09122c721cfcdc3553.jpeg)

Author

[![Nimrod Kramer](https://cdn.prod.website-files.com/5e0f1144930a8bc8aace526c/64b78e6fd75bbfcf43f84516_nimrod-square__3__720.png)

Nimrod Kramer

@NimrodKramer](https://app.daily.dev/kramer)

Related tags on daily.dev

![toc](https://cdn.prod.website-files.com/5e0a5d9d743608d0f3ea6753/60aa101034a8ea4254921ef1_Unread.svg)

Table of contents

![arrow-down](https://cdn.prod.website-files.com/5e0a5d9d743608d0f3ea6753/60aa1092c2681854cb7d9e94_Arrow%20Down.svg)

[Read more on daily.dev

![open link icon](https://cdn.prod.website-files.com/5e0a5d9d743608d0f3ea6753/65fc45c5f1fa2162fa715b91__Open%20Link_.svg)](https://app.daily.dev/?utm_source=landing&utm_medium=blog&utm_campaign=toc)

🎯

Discover the capabilities of OpenAI's GPT-4o, including processing text, images, and audio with improved accuracy and ethical guardrails for AI interactions.

[GPT-4o](https://en.wikipedia.org/wiki/GPT-4o) is [OpenAI](https://openai.com/)'s latest and most advanced [multimodal large language model](https://daily.dev/blog/mm1-the-advanced-30b-parameters-multimodal-llm-from-apple#table-comparing-models). It can process and generate text, images, and audio, enabling seamless human-computer interaction across various modalities. GPT-4o boasts improved capabilities over its predecessors in language understanding, image processing, and audio generation.

**Key Features:**

* **Multimodal Inputs & Outputs:** Accepts text, audio, and image inputs; generates text, audio, and image outputs
* **Natural Language Processing:** Engages in human-like conversations, answers complex questions, generates high-quality content
* **Vision Capabilities:** Analyzes images, charts, and diagrams; describes visual elements; generates new images
* **Audio Capabilities:** Speech recognition, text-to-speech conversion, audio analysis and generation
* **Real-time Conversation:** Enables real-time, back-and-forth conversations across multiple modalities
* **Multilingual Support:** Understands and generates content in over 50 languages
* **Contextual Awareness:** Provides relevant and coherent responses based on user intent, background knowledge, and conversational history
* **Ethical Guardrails:** Ensures responsible, unbiased, and factually accurate outputs

**Accessing GPT-4o:**

| User Type | Access Method |
| --- | --- |
| Current [ChatGPT](https://openai.com/chatgpt) Plus/Team Users | Upgrade within the ChatGPT interface |
| Developers | Access via the GPT-4o API with higher rate limits and reduced costs |

**Limitations & Responsible Use:**

While powerful, GPT-4o has limitations and potential risks, such as providing inaccurate information, perpetuating biases, and being vulnerable to exploitation. Safe and ethical usage is crucial, including being aware of its boundaries, verifying outputs, and prioritizing transparency and accountability.

**The [Future of GPT-4o](https://app.daily.dev/posts/emxPieNtY):**

Upcoming enhancements include expanded multimodal capabilities, improved accuracy, and increased efficiency. GPT-4o is expected to have a significant impact on the AI market, enabling new applications and transforming industries.

## Related video from YouTube

## Understanding [GPT-4o](https://en.wikipedia.org/wiki/GPT-4o)'s Capabilities

![GPT-4o](https://cdn.prod.website-files.com/5e0f1144930a8bc8aace526c/66437e91e15f54e26397244a_en.wikipedia.org-c493eb0bd3d3bb2621f8538ee794b396.jpeg)

### GPT-4o's Text, Voice, and Vision Skills

GPT-4o is a powerful multimodal model that combines text, audio, and visual inputs and outputs. It can understand and generate human-like language, process and generate images, and comprehend and produce audio with high accuracy and speed.

**Text Capabilities**

GPT-4o can engage in natural conversations, answer complex questions, and generate high-quality content across various domains. Its language understanding and generation are on par with human-level proficiency, making interactions feel effortless and intuitive.

**Vision Capabilities**

GPT-4o can analyze and interpret images, charts, and diagrams with precision. It can describe visual elements in detail, identify objects and patterns, and even generate new images based on textual prompts.

**Audio Capabilities**

GPT-4o can process and generate audio data, including speech recognition, text-to-speech conversion, and audio analysis. This feature facilitates natural voice interactions, making it ideal for virtual assistants, audio content creation, and accessibility applications.

### New Features in GPT-4o

GPT-4o introduces several innovative features that enhance the user experience and expand its potential applications:

| Feature | Description |
| --- | --- |
| Real-time Conversation | Engage in real-time, back-and-forth conversations across multiple modalities |
| Improved Multilingual Support | Understand and generate content in over 50 languages |
| Multimodal Generation | Generate outputs that combine text, images, and audio |
| Contextual Awareness | Provide more relevant and coherent responses based on user intent, background knowledge, and conversational history |
| Enhanced Safety and Ethical Guardrails | Ensure responsible, unbiased, and factually accurate outputs |

With its remarkable capabilities and innovative features, GPT-4o represents a significant leap forward in the field of artificial intelligence, paving the way for more natural and intuitive human-machine interactions across a wide range of applications.

## Accessing GPT-4o

GPT-4o is easy to access, with options for different user tiers and needs.

### Upgrading to GPT-4o for Current Users

If you're already a [ChatGPT Plus](https://app.daily.dev/posts/0bF7WJmyo) or Team subscriber, you can upgrade to GPT-4o to take advantage of its enhanced capabilities. To do so, follow these steps:

1. Click on the dropdown menu at the top of the page. 2. Select GPT-4o from the list of available models.

As a Plus or Team user, you'll enjoy higher message limits and faster response times compared to the free tier.

### GPT-4o API for Developers

Developers can access GPT-4o through the API, which offers several benefits:

| **Feature** | **Description** |
| --- | --- |
| Revised rate limits | 5x higher rate limits compared to GPT-4 Turbo |
| Cost-efficiency | 50% reduction in costs compared to GPT-4 Turbo |
| Enhanced capabilities | Advanced multimodal capabilities, including text, voice, and vision inputs and outputs |

To get started with the GPT-4o API:

1. Sign up for an OpenAI account. 2. Obtain an API key. 3. Follow the API documentation to integrate GPT-4o into your projects.

## Using GPT-4o in Practice

GPT-4o offers a wide range of applications in professional settings, from data analysis and decision support to content creation and language tasks. In this section, we'll explore the numerous ways GPT-4o can be used in practice, providing examples pertinent to the development and networking context.

### Data Analysis and Decision Support

GPT-4o can be used to analyze large datasets, identify patterns, and provide objective overviews. This enables informed decision-making and improves customer experiences.

| **Capability** | **Description** |
| --- | --- |
| Data Analysis | Analyze large datasets to identify patterns and trends |
| Objective Overviews | Provide objective overviews and comparisons of different solutions |
| Scoring | Assign scores to each solution based on available data |
| Summarization | Summarize pros and cons of each option, enabling informed decisions |

For instance, in a marketing workflow, GPT-4o can help evaluate solutions, attribute scores, and provide a comprehensive overview of the decision-making process. This enables marketers to make data-driven decisions, saving time and resources.

### Content Creation and Language Tasks

GPT-4o's generative powers can aid in content creation, language translation, and supporting a multilingual workforce.

| **Capability** | **Description** |
| --- | --- |
| Content Generation | Generate high-quality content in a matter of seconds |
| Language Translation | Translate text from one language to another with precision |
| Creative Writing | Create poems, stories, and screenplays with originality and coherence |
| Writing Assistance | Assist in writing tasks, such as copywriting and creative writing |

For example, in a content creation workflow, GPT-4o can help generate blog posts, social media captions, or product descriptions, freeing up time for more strategic tasks. Its language translation capabilities can also facilitate communication across language barriers, enabling global collaboration and expansion.

By leveraging GPT-4o's capabilities in practice, professionals can streamline their workflows, enhance decision-making, and drive innovation in their respective fields.

###### sbb-itb-bfaad5b

## Limitations and Responsible Use

GPT-4o, like its predecessors, has limitations and potential risks. It's essential to understand these boundaries and take steps to mitigate them, ensuring safe and ethical usage.

### GPT-4o's Boundaries

GPT-4o is not perfect and can make mistakes. It may:

* Provide inaccurate information
* Perpetuate biases and stereotypes
* Create "hallucinations" (information that is not based on facts)
* Lack human knowledge and skills in certain domains
* Be vulnerable to exploitation and social engineering

OpenAI has taken steps to address these issues, but users must also be aware of these limitations and take steps to verify the accuracy of the model's outputs.

### Safe and Ethical Usage

To ensure safe and ethical usage of GPT-4o, follow these guidelines:

* Use the model for its intended purposes
* Be aware of its limitations
* Take steps to mitigate potential risks
* Prioritize transparency, accountability, and responsibility

Developers, businesses, and users must work together to establish clear guidelines and regulations for the responsible development and deployment of AI models like GPT-4o.

By understanding GPT-4o's limitations and taking steps to mitigate them, we can ensure that the model is used in a way that benefits society as a whole.

## The Future of GPT-4o

GPT-4o's future is promising, with upcoming updates and improvements set to further transform the AI landscape. As the technology evolves, we can expect new use cases to emerge and existing ones to become even more sophisticated.

### Upcoming Enhancements

OpenAI has hinted at several upcoming enhancements to GPT-4o, including:

| Enhancement | Description |
| --- | --- |
| Expanded Multimodal Capabilities | Enable more advanced applications, such as realistic AI-generated videos and images |
| Improved Accuracy | Increase the model's precision and reliability |
| Increased Efficiency | Allow for faster processing and response times |

These updates are expected to enable even more advanced applications, such as more realistic AI-generated videos and images, and more accurate language translation.

### Potential Market Impact

The future of GPT-4o is likely to have a significant impact on the AI market, as it continues to push the boundaries of what is possible with language models. As the technology becomes more advanced and widely adopted, we can expect to see:

* New market leaders emerge
* Existing ones adapt to the changing landscape
* Significant changes in the way businesses operate
* Changes in the way people interact with technology

Overall, the future of GPT-4o is exciting and full of potential, with the possibility of transforming industries and revolutionizing the way we interact with technology.

## Conclusion

GPT-4o is a significant advancement in AI and natural language processing. With its advanced multimodal capabilities, increased accuracy, and improved efficiency, this powerful language model opens up new possibilities for developers and businesses.

### Key Points

* GPT-4o can process text, images, and voice inputs, enabling various applications across industries.
* Upcoming enhancements will further improve the model's capabilities.
* GPT-4o has the potential to transform industries and revolutionize how we interact with technology.

### Recommendations for Developers

To fully leverage GPT-4o, consider the following:

| **Recommendation** | **Description** |
| --- | --- |
| Integrate GPT-4o into workflows | Explore ways to incorporate GPT-4o into development processes, such as code generation, natural language processing, and data analysis. |
| Leverage multimodal capabilities | Take advantage of GPT-4o's ability to process text, images, and voice inputs to create innovative applications. |
| Prioritize responsible and ethical use | Ensure applications adhere to best practices for privacy, security, and ethical considerations. |
| Stay up-to-date with advancements | Stay informed about the latest developments, updates, and best practices to leverage the full potential of GPT-4o. |

By embracing GPT-4o and its capabilities, developers can unlock new opportunities for innovation, streamline workflows, and create groundbreaking applications that push the boundaries of AI and natural language processing.

## FAQs

### What can GPT-4o do?

GPT-4o is a powerful language model that can process and generate human-like text, images, and audio. It can solve written problems, generate original content, and even create art.

### What are the features of GPT-4o?

GPT-4o has several features that make it more advanced than its predecessors. These include:

| Feature | Description |
| --- | --- |
| Multimodal capabilities | Accepts text, images, and audio inputs and generates outputs in various formats |
| Long-form content creation | Can generate content up to 25,000 words, making it suitable for long-form writing and document analysis |
| Image analysis | Can analyze and generate captions for images |

### What are the limitations of GPT-4o?

GPT-4o is not perfect and has some limitations. These include:

| Limitation | Description |
| --- | --- |
| Lack of accuracy | May generate inaccurate or nonsensical responses |
| Inflammatory content | May produce inflammatory or offensive content due to its training on internet data |

### Is GPT-4o better than GPT-3?

GPT-4o is faster and more accurate than GPT-3. It has several improvements that make it more powerful and efficient.

## Related Blog Posts

* [OpenAI Q: Everything you need to know in one place](/blog/openai-q-everything-you-need-to-know-in-one-place/)
* [MM1: The Advanced 30B Parameters Multimodal LLM from Apple](/blog/mm1-the-advanced-30b-parameters-multimodal-llm-from-apple/)
* [Sora by OpenAI: Is it a Game-Changer for AI-Generated Videos?](/blog/sora-by-openai-is-it-a-game-changer-for-ai-generated-videos/)
* [Google Cloud Next 2024: All the announcements in one place](/blog/google-cloud-next-2024-all-the-announcements-in-one-place/)

Author

[![Nimrod Kramer](https://cdn.prod.website-files.com/5e0f1144930a8bc8aace526c/64b78e6fd75bbfcf43f84516_nimrod-square__3__720.png)](https://app.daily.dev/kramer)

Related tags on daily.dev

![toc](https://cdn.prod.website-files.com/5e0a5d9d743608d0f3ea6753/60aa101034a8ea4254921ef1_Unread.svg)

Table of contents

[Read more on daily.dev

![open link icon](https://cdn.prod.website-files.com/5e0a5d9d743608d0f3ea6753/65fc45c5f1fa2162fa715b91__Open%20Link_.svg)](https://api.daily.dev/get)

[![daily.dev platform](https://cdn.prod.website-files.com/5e0a5d9d743608d0f3ea6753/65fc443cac134dd95826e584_daily.dev.jpg)

Why not level up your reading with daily.dev?

Stay up-to-date with the latest developer news every time you open a new tab.

Start reading - Free forever](https://api.daily.dev/get)

## Why not level up your reading with daily.dev?

Stay up-to-date with the latest developer news every time you open a new tab.

![browsers logos](https://cdn.prod.website-files.com/5e0a5d9d743608d0f3ea6753/65802a0f2f75540ff5e8c82c_Vectors-Wrapper.svg)[Start reading - Free forever](https://api.daily.dev/get)

## Read more

[![Ultimate Guide to Cross-Platform Testing Frameworks](https://cdn.prod.website-files.com/5e0f1144930a8bc8aace526c/68415d8c85f303451e16e365_6811696992ebe69020265270-1745974490923.jpeg)

Get into tech

Ultimate Guide to Cross-Platform Testing Frameworks

Learn how to select, integrate, and troubleshoot cross-platform testing frameworks to enhance software quality across multiple devices.

![Nimrod Kramer](https://cdn.prod.website-files.com/5e0f1144930a8bc8aace526c/64b78e6fd75bbfcf43f84516_nimrod-square__3__720.png)

Nimrod Kramer

April 30, 2025](/blog/ultimate-guide-to-cross-platform-testing-frameworks)

[![How to become the ultimate vibe coder](https://cdn.prod.website-files.com/5e0f1144930a8bc8aace526c/68415be67e9a9597dea6ee00_68414e091bd3e22313035ab3-1749112451760.jpeg)

Get into tech

How to become the ultimate vibe coder

Explore vibe coding, a blend of creativity and technical skills that enhances productivity and fosters collaboration in software development.

![Nimrod Kramer](https://cdn.prod.website-files.com/5e0f1144930a8bc8aace526c/64b78e6fd75bbfcf43f84516_nimrod-square__3__720.png)

Nimrod Kramer

June 5, 2025](/blog/how-to-become-the-ultimate-vibe-coder)

[![Building the developer economy at daily.dev](https://cdn.prod.website-files.com/5e0f1144930a8bc8aace526c/6810d64a6164eec9970a4126_cores.jpg)

daily.dev

Building the developer economy at daily.dev

Discover how daily.dev is building a new developer economy powered by Cores and Awards — where real skills, contributions, and craftsmanship are recognized and rewarded by the community.

![Nimrod Kramer](https://cdn.prod.website-files.com/5e0f1144930a8bc8aace526c/64b78e6fd75bbfcf43f84516_nimrod-square__3__720.png)

Nimrod Kramer

April 29, 2025](/blog/building-the-developer-economy-at-daily-dev)

[See more on daily.dev](https://api.daily.dev/get)

[![daily.dev logo](https://cdn.prod.website-files.com/5e0a5d9d743608d0f3ea6753/5f1d8b1b9c7814aae6b69044_Daily%20Full%20logo.svg)](/?r=0)

daily.dev is a professional network for developers to learn, collaborate, and grow together.

[![daily.dev on App Store](https://cdn.prod.website-files.com/5e0a5d9d743608d0f3ea6753/67a1d747a33f5e26c766ecfd_Download_on_the_App_Store_Badge_US-UK_RGB_wht_092917.svg)](https://apps.apple.com/app/daily-dev/id6740634400)[![daily.dev on Google Play](https://cdn.prod.website-files.com/5e0a5d9d743608d0f3ea6753/67a1e56f9bb1080f810aca8c_GetItOnGooglePlay_Badge_Web_color_English%201.svg)](https://play.google.com/store/apps/details?id=dev.daily)

[![Twitter](https://cdn.prod.website-files.com/5e0a5d9d743608d0f3ea6753/66e174de6a17acea46d7f208_dailydev.svg)](https://app.daily.dev/squads/daily_updates)[![Twitter](https://cdn.prod.website-files.com/5e0a5d9d743608d0f3ea6753/652013281659e3772def4d45_logo.svg)](https://twitter.com/dailydotdev)[![GitHub](https://cdn.prod.website-files.com/5e0a5d9d743608d0f3ea6753/5e19de93a7d3a2e0db2e086a_github.svg)](https://github.com/dailydotdev/daily)[![Instagram](https://cdn.prod.website-files.com/5e0a5d9d743608d0f3ea6753/5e4e39ab47c62082de2e8889_Instagram.svg)](https://www.instagram.com/dailydotdev/)[![Instagram](https://cdn.prod.website-files.com/5e0a5d9d743608d0f3ea6753/6345cbe66cf515e13bf26193_White.svg)](https://www.tiktok.com/%40dailydotdev)

Product

[Chrome extension](https://r.daily.dev/chrome) [Edge add-on](https://microsoftedge.microsoft.com/addons/detail/dailydev-the-homepage-/cbdhgldgiancdheindpekpcbkccpjaeb?hl=en-GB)[Firefox add-ons](https://addons.mozilla.org/en-US/firefox/addon/daily/)[iOS app](https://apps.apple.com/app/daily-dev/id6740634400)[Android app](https://play.google.com/store/apps/details?id=dev.daily)[Web version](https://app.daily.dev/?utm_source=landing&utm_medium=cta&utm_campaign=landing_conversions)[Changelog](https://app.daily.dev/squads/daily_updates)[Status](https://www.dailydevstatus.com/)

Community

[Docs](https://docs.daily.dev/docs/intro)[Open source](https://github.com/dailydotdev/daily)[Feature requests](https://github.com/dailydotdev/daily/discussions)[Online events](https://www.meetup.com/the-monthly-dev-world-class-talks-by-expert-developers/)[Swag store](https://store.daily.dev/)

Company

[Careers](/careers)[Blog](/blog-superold)[Advertise](https://business.daily.dev/?ref=homepage)[Brand book](https://brand.daily.dev/)[About us](/about-us)[Contact](/contact)

© 2025 Daily Dev Ltd.

[Terms](/tos)[Privacy](/privacy)[Guidelines](https://r.daily.dev/content-guidelines)

Working remotely wherever we're happiest

🇮🇱 🇮🇹 🇵🇭 🇳🇱 🇬🇧 🇭🇷 🇱🇹 🇦🇺 🇵🇱 🇳🇴 🇦🇱 🇵🇹

![](https://px.ads.linkedin.com/collect/?pid=3001529&fmt=gif)

# **Build a Weather Tool**

In [53]:
import os
from getpass import getpass

os.environ["WEATHER_API_KEY"] = getpass("Enter your weather API Key: ")
WEATHER_API_KEY = os.environ["WEATHER_API_KEY"]

Enter your weather API Key: ··········


In [54]:
import requests

@tool
def get_weather(query: str) -> list:
    """Search weatherapi to get the current weather."""
    base_url = "http://api.weatherapi.com/v1/current.json"
    complete_url = f"{base_url}?key={WEATHER_API_KEY}&q={query}"

    response = requests.get(complete_url)
    data = response.json()
    if data.get("location"):
        return data
    else:
        return "Weather Data Not Found"

In [56]:
get_weather.invoke("Bangalore")

{'location': {'name': 'Bangalore',
  'region': 'Karnataka',
  'country': 'India',
  'lat': 12.9833,
  'lon': 77.5833,
  'tz_id': 'Asia/Kolkata',
  'localtime_epoch': 1751270688,
  'localtime': '2025-06-30 13:34'},
 'current': {'last_updated_epoch': 1751270400,
  'last_updated': '2025-06-30 13:30',
  'temp_c': 29.2,
  'temp_f': 84.6,
  'is_day': 1,
  'condition': {'text': 'Partly cloudy',
   'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png',
   'code': 1003},
  'wind_mph': 18.6,
  'wind_kph': 29.9,
  'wind_degree': 270,
  'wind_dir': 'W',
  'pressure_mb': 1011.0,
  'pressure_in': 29.85,
  'precip_mm': 0.0,
  'precip_in': 0.0,
  'humidity': 58,
  'cloud': 50,
  'feelslike_c': 29.3,
  'feelslike_f': 84.7,
  'windchill_c': 29.1,
  'windchill_f': 84.4,
  'heatindex_c': 29.2,
  'heatindex_f': 84.5,
  'dewpoint_c': 14.6,
  'dewpoint_f': 58.3,
  'vis_km': 8.0,
  'vis_miles': 4.0,
  'uv': 9.9,
  'gust_mph': 21.4,
  'gust_kph': 34.4}}

In [57]:
import rich

result = get_weather.invoke("Zurich")
rich.print_json(data=result)

{
  "location": {
    "name": "Zurich",
    "region": "",
    "country": "Switzerland",
    "lat": 47.3667,
    "lon": 8.55,
    "tz_id": "Europe/Zurich",
    "localtime_epoch": 1751269718,
    "localtime": "2025-06-30 09:48"
  },
  "current": {
    "last_updated_epoch": 1751269500,
    "last_updated": "2025-06-30 09:45",
    "temp_c": 23.3,
    "temp_f": 73.9,
    "is_day": 1,
    "condition": {
      "text": "Partly Cloudy",
      "icon": "//cdn.weatherapi.com/weather/64x64/day/116.png",
      "code": 1003
    },
    "wind_mph": 3.6,
    "wind_kph": 5.8,
    "wind_degree": 28,
    "wind_dir": "NNE",
    "pressure_mb": 1020.0,
    "pressure_in": 30.12,
    "precip_mm": 0.0,
    "precip_in": 0.0,
    "humidity": 69,
    "cloud": 0,
    "feelslike_c": 25.1,
    "feelslike_f": 77.2,
    "windchill_c": 24.5,
    "windchill_f": 76.1,
    "heatindex_c": 26.0,
    "heatindex_f": 78.7,
    "dewpoint_c": 17.5,
    "dewpoint_f": 63.5,
    "vis_km": 10.0,
    "vis_miles": 6.0,
    "uv": 2.5,
    "gust_mph": 4.1,
    "gust_kph": 6.6
  }
}

# **Explore LLM tool calling with custom tools**


# Tool calling for LLMs with native support for tool or function calling

In [58]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model = "gemini-1.5-flash-latest")

In [59]:
tools = [multiply, search_web_extract_info, get_weather]
chatgpt_with_tools = llm.bind_tools(tools)

In [60]:
prompt = """
            Given only the tools at your disposal, mention tool calls for the following tasks:
            Do not change the query given for any search tasks
            1. What is 2.1 times 3.5
            2. What is the current weather in Greenland today
            3. What are the 4 major Agentic AI Design Patterns
         """

results = chatgpt_with_tools.invoke(prompt)

In [61]:
results

AIMessage(content="Here's how to accomplish each task using the available tools:\n\n1. **Calculate 2.1 times 3.5:**", additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 2.1, "b": 3.5}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash-latest', 'safety_ratings': []}, id='run--ee4fbe56-0c55-4c91-b2c3-5c05932f680d-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2.1, 'b': 3.5}, 'id': '1aa4766b-a8c9-48d5-b0ec-462240af76fd', 'type': 'tool_call'}], usage_metadata={'input_tokens': 131, 'output_tokens': 34, 'total_tokens': 165, 'input_token_details': {'cache_read': 0}})

In [62]:
results.tool_calls

[{'name': 'multiply',
  'args': {'a': 2.1, 'b': 3.5},
  'id': '1aa4766b-a8c9-48d5-b0ec-462240af76fd',
  'type': 'tool_call'}]

In [63]:
multiply

StructuredTool(name='multiply', description='use to multiply numbers', args_schema=<class '__main__.CalculatorInput'>, return_direct=True, func=<function multiply at 0x7f0ce8fc3f60>)

In [64]:
toolkit = {
    "multiply": multiply,
    "search_web_extract_info": search_web_extract_info,
    "get_weather": get_weather
}

for tool_call in results.tool_calls:
    selected_tool = toolkit[tool_call["name"].lower()]
    print(f"Calling tool: {tool_call['name']}")
    tool_output = selected_tool.invoke(tool_call["args"])
    print(tool_output)
    print()

Calling tool: multiply
7.3500000000000005



In [65]:
tools

[StructuredTool(name='multiply', description='use to multiply numbers', args_schema=<class '__main__.CalculatorInput'>, return_direct=True, func=<function multiply at 0x7f0ce8fc3f60>),
 StructuredTool(name='search_web_extract_info', description='Search the web for a query and extracts useful information from the search links', args_schema=<class 'langchain_core.utils.pydantic.search_web_extract_info'>, func=<function search_web_extract_info at 0x7f0cd432eac0>),
 StructuredTool(name='get_weather', description='Search weatherapi to get the current weather.', args_schema=<class 'langchain_core.utils.pydantic.get_weather'>, func=<function get_weather at 0x7f0ccfe98360>)]

# Tool calling for LLMs without native support for tool or function calling

In [66]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import render_text_description

rendered_tools = render_text_description(tools)
print(rendered_tools)

multiply(a: float, b: float) -> float - use to multiply numbers
search_web_extract_info(query: str) -> list - Search the web for a query and extracts useful information from the search links
get_weather(query: str) -> list - Search weatherapi to get the current weather.


In [67]:
system_prompt = f"""\
You are an assistant that has access to the following set of tools.
Here are the names and descriptions for each tool:

{rendered_tools}

Given the user instructions, for each instruction do the following:
 - Return the name and input of the tool to use.
 - Return your response as a JSON blob with 'name' and 'arguments' keys.
 - The `arguments` should be a dictionary, with keys corresponding
   to the argument names and the values corresponding to the requested values.
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", "{input}")
    ]
)

In [68]:
instructions = [
                  {"input" : "What is 2.1 times 3.5"},
                  {"input" : "What is the current weather in Greenland"},
                  {"input" : "Tell me about the current state of Agentic AI in the industry" }
               ]

In [70]:
from langchain_core.output_parsers import JsonOutputParser

chain = (prompt
            |
         llm
            |
         JsonOutputParser())

In [71]:
responses = chain.map().invoke(instructions)

In [72]:
responses

[{'name': 'multiply', 'arguments': {'a': 2.1, 'b': 3.5}},
 {'name': 'get_weather', 'arguments': {'query': 'Greenland'}},
 {'name': 'search_web_extract_info',
  'arguments': {'query': 'Current state of Agentic AI in the industry'}}]

In [73]:
toolkit = {
    "multiply": multiply,
    "search_web_extract_info": search_web_extract_info,
    "get_weather": get_weather
}

for tool_call in responses:
    selected_tool = toolkit[tool_call["name"].lower()]
    print(f"Calling tool: {tool_call['name']}")
    tool_output = selected_tool.invoke(tool_call["arguments"])
    print(tool_output)
    print()

Calling tool: multiply
7.3500000000000005

Calling tool: get_weather
{'location': {'name': 'Nuuk', 'region': 'Vestgronland', 'country': 'Greenland', 'lat': 64.183, 'lon': -51.75, 'tz_id': 'America/Nuuk', 'localtime_epoch': 1751271077, 'localtime': '2025-06-30 07:11'}, 'current': {'last_updated_epoch': 1751270400, 'last_updated': '2025-06-30 07:00', 'temp_c': 5.1, 'temp_f': 41.2, 'is_day': 0, 'condition': {'text': 'Light rain', 'icon': '//cdn.weatherapi.com/weather/64x64/night/296.png', 'code': 1183}, 'wind_mph': 9.8, 'wind_kph': 15.8, 'wind_degree': 137, 'wind_dir': 'SE', 'pressure_mb': 1000.0, 'pressure_in': 29.53, 'precip_mm': 1.25, 'precip_in': 0.05, 'humidity': 100, 'cloud': 100, 'feelslike_c': 1.8, 'feelslike_f': 35.2, 'windchill_c': -1.2, 'windchill_f': 29.9, 'heatindex_c': 2.7, 'heatindex_f': 36.9, 'dewpoint_c': 2.1, 'dewpoint_f': 35.9, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 0.0, 'gust_mph': 20.7, 'gust_kph': 33.3}}

Calling tool: search_web_extract_info


 20%|██        | 1/5 [00:00<00:01,  3.96it/s]

Extraction blocked for url:  https://www.theregister.com/2025/06/29/ai_agents_fail_a_lot/


100%|██████████| 5/5 [00:04<00:00,  1.19it/s]

['The Agentic AI Applications and Future Trends beyond 2025\n[Techpilot\n![Tech Pilot AI Logo](https://techpilot.ai/wp-content/themes/easyfin-theme/assets/images/logo.svg)](https://techpilot.ai)\n\n* [Tools](https://techpilot.ai/tools/)\n* [Services](https://techpilot.ai/services/)\n* [Blog](https://techpilot.ai/category/blog/)\n* [About Us](https://techpilot.ai/about-us/)\n* [Contact](https://techpilot.ai/contact/)\n\n[Home](https://techpilot.ai)\n»\n[AI Agents](https://techpilot.ai/category/ai-agents/)» The Agentic AI Applications and Future Trends\n\n# The Agentic AI Applications and Future Trends\n\nWhat are the current possible Agentic AI Applications in practice and what are the future trends that will shape the landscape beyond 2025.\n\n![Marketing & Tech](https://techpilot.ai/wp-content/uploads/2023/03/tp.png)\n\n![future-of-ai-agents](https://techpilot.ai/wp-content/uploads/2025/06/future-of-ai-agents-1024x683.jpg)\n\nPublished: June 29, 2025 | Updated: June 29, 2025\n\nAgenti

In [74]:
for doc in tool_output:
    print(doc)
    print()

The Agentic AI Applications and Future Trends beyond 2025
[Techpilot
![Tech Pilot AI Logo](https://techpilot.ai/wp-content/themes/easyfin-theme/assets/images/logo.svg)](https://techpilot.ai)

* [Tools](https://techpilot.ai/tools/)
* [Services](https://techpilot.ai/services/)
* [Blog](https://techpilot.ai/category/blog/)
* [About Us](https://techpilot.ai/about-us/)
* [Contact](https://techpilot.ai/contact/)

[Home](https://techpilot.ai)
»
[AI Agents](https://techpilot.ai/category/ai-agents/)» The Agentic AI Applications and Future Trends

# The Agentic AI Applications and Future Trends

What are the current possible Agentic AI Applications in practice and what are the future trends that will shape the landscape beyond 2025.

![Marketing & Tech](https://techpilot.ai/wp-content/uploads/2023/03/tp.png)

![future-of-ai-agents](https://techpilot.ai/wp-content/uploads/2025/06/future-of-ai-agents-1024x683.jpg)

Published: June 29, 2025 | Updated: June 29, 2025

Agentic AI applications streamli